In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import sklearn
from shapely.geometry import Point
import seaborn as sns

# MAKING CLEAN SAS CSV, ONLY RUN FIRST TWO IF YOU ALREADY HAVE CLEAN CSV

In [2]:
sas_path = r"C:\Users\Usuario\Desktop\TUE\Data Challenge 2\data\stop_and_search.csv"
shape_path = r'C:\Users\Usuario\Desktop\TUE\Data Challenge 2\data\Shapefiles\statistical-gis-boundaries-london\ESRI\LSOA_2011_London_gen_MHW.shp'

In [3]:
df_sas = pd.read_csv(sas_path)
shape = gpd.read_file(shape_path)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1752\872919928.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sas = pd.read_csv(sas_path)


In [ ]:
london_boroughs = [
    "Barking and Dagenham", "Barnet", "Bexley", "Brent", "Bromley",
    "Camden", "Croydon", "Ealing", "Enfield", "Greenwich", "Hackney",
    "Hammersmith and Fulham", "Haringey", "Harrow", "Havering",
    "Hillingdon", "Hounslow", "Islington", "Kensington and Chelsea",
    "Kingston upon Thames", "Lambeth", "Lewisham", "Merton", "Newham",
    "Redbridge", "Richmond upon Thames", "Southwark", "Sutton",
    "Tower Hamlets", "Waltham Forest", "Wandsworth", "Westminster"
]


In [ ]:
df_sas[df_sas["Longitude"].notna()]

In [ ]:
shape

In [ ]:
df_sas = df_sas.drop(columns = ["Part of a policing operation","Policing operation","Outcome linked to object of search","Removal of more than just outer clothing"])
df_sas = df_sas.dropna(subset=['Latitude'])

In [ ]:
# Read the CSV data
df_sas = pd.read_csv(sas_path)

# Create a GeoDataFrame from df_sas with the appropriate coordinate system
geometry = [Point(xy) for xy in zip(df_sas['Longitude'], df_sas['Latitude'])]
geo_df_sas = gpd.GeoDataFrame(df_sas, geometry=geometry, crs="EPSG:4326")

# Read the shapefile
shape = gpd.read_file(shape_path)

# If the CRS of shape is different, transform geo_df_sas to match it
if geo_df_sas.crs != shape.crs:
    geo_df_sas = geo_df_sas.to_crs(shape.crs)

# Perform a spatial join to find which borough each point falls into
joined_df = gpd.sjoin(geo_df_sas, shape, how="left", op="within")

# Add the borough name from the 'LAD11NM' attribute to df_sas
df_sas['LSOA'] = joined_df['LSOA11CD']

# Optionally, you might want to check and handle any points that didn't match a borough
df_sas['LSOA'].fillna('LSOA not found', inplace=True)

# df_sas now contains an additional column "Borough" with the borough name for each entry


In [ ]:
df_sas

In [ ]:
# Count of "Borough not found" instances
borough_not_found_count = df_sas['LSOA'].value_counts().get('LSOA not found', 0)

print("Count of 'LSOA not found' instances:", borough_not_found_count)


In [ ]:
df_sas_with_LSOA = df_sas[df_sas["LSOA"] != "LSOA not found"]
df_sas_LSOA_grouped = df_sas_with_LSOA.groupby("LSOA").size().reset_index(name='count')
df_sas_LSOA_grouped

In [ ]:
merged_gdf = shape.merge(df_sas_LSOA_grouped, left_on='LSOA11CD', right_on='LSOA')

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
merged_gdf.plot(column='count', cmap='coolwarm', linewidth=0, ax=ax, edgecolor='1', legend=True)

plt.title('Percentage of people being missidentified')
plt.show()

In [ ]:
LSOA# Remove rows where 'Borough' is 'Borough not found' or not in list
df_sas = df_sas[df_sas['LSOA'] != 'LSOA not found']


In [ ]:
df_sas['Date'] = pd.to_datetime(df_sas['Date'])

# Extract year and month from the 'Date' column
df_sas['year'] = df_sas['Date'].dt.year
df_sas['month'] = df_sas['Date'].dt.month
df_sas['day'] = df_sas['Date'].dt.day
df_sas['hour'] = df_sas['Date'].dt.hour

In [ ]:
df_sas.info()

In [ ]:
df_sas.to_csv("stop_and_search_data_with_borough.csv")

# LOADING CLEAN STOP AND SEARCH

In [ ]:
sas_clean_path = r"C:\Users\Usuario\Desktop\TUE\Data Challenge 2\data\stop_and_search_data_with_borough.csv"
df_sas_clean = pd.read_csv(sas_clean_path)

In [ ]:
df_sas_clean

## Removing 2024 since not relevant

In [ ]:
df_sas_clean = df_sas_clean[df_sas_clean['year'] != 2024]

In [ ]:
df_sas_year_bor_count = df_sas_clean.groupby(['Borough', 'year']).size().reset_index(name='Count')

# Display the new DataFrame to confirm it's correctly created
df_sas_year_bor_count

In [ ]:
fig, axes = plt.subplots(8, 4, figsize=(20, 20), sharey=False, sharex=False)

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Loop through each borough and plot on its respective subplot
for i, borough in enumerate(london_boroughs):
    borough_data = df_sas_year_bor_count[df_sas_year_bor_count['Borough'] == borough]
    ax = axes[i]
    sns.lineplot(data=borough_data, x='year', y='Count', ax=ax)
    ax.set_title(borough)
    ax.set_xlabel('Year')
    ax.set_ylabel('Count')

# Remove any empty subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust the layout
plt.tight_layout()
plt.show()

In [ ]:
# Pivot the dataframe to get counts for each year in separate columns
df_pivot = df_sas_year_bor_count.pivot(index='Borough', columns='year', values='Count').reset_index()

# Calculate the change and percentage change from 2016 to 2023
df_pivot['change'] = df_pivot[2023] - df_pivot[2016]
df_pivot['percentage_change'] = (df_pivot['change'] / df_pivot[2016]) * 100

# Create a new dataframe with the boroughs and their respective percentage changes
df_percentage_change = df_pivot[['Borough', 'percentage_change']]

# Display the resulting dataframe
print(df_percentage_change)

In [ ]:
df_percentage_change["percentage_change"].median(), df_percentage_change["percentage_change"].mean()

# PAS DATA

In [4]:
pas_path_1 = r"C:\Users\Usuario\Desktop\TUE\Data Challenge 2\data\pas_data_ward_level\PAS_ward_level_FY_15_17.csv"
pas_path_2 = r"C:\Users\Usuario\Desktop\TUE\Data Challenge 2\data\pas_data_ward_level\PAS_ward_level_FY_17_18.csv"
pas_path_3 = r"C:\Users\Usuario\Desktop\TUE\Data Challenge 2\data\pas_data_ward_level\PAS_ward_level_FY_18_19.csv"
pas_path_4 = r"C:\Users\Usuario\Desktop\TUE\Data Challenge 2\data\pas_data_ward_level\PAS_ward_level_FY_19_20.csv"
pas_path_5 = r"C:\Users\Usuario\Desktop\TUE\Data Challenge 2\data\pas_data_ward_level\PAS_ward_level_FY_20_21.csv"

In [5]:
df_pas1 = pd.read_csv(pas_path_1)
df_pas2 = pd.read_csv(pas_path_2)
df_pas3 = pd.read_csv(pas_path_3)
df_pas4 = pd.read_csv(pas_path_4)
df_pas5 = pd.read_csv(pas_path_5)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1752\3828281856.py:1: DtypeWarning: Columns (13,14,15,16,17,18,19,20,21,22,35,177,410,411,412,415,416,417,418,419,420,421,422,423) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pas1 = pd.read_csv(pas_path_1)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_1752\3828281856.py:2: DtypeWarning: Columns (30,31,32,98,99,100,101,102,103,104,105,125,126,127,199,200) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pas2 = pd.read_csv(pas_path_2)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_1752\3828281856.py:3: DtypeWarning: Columns (25,26,27,28,108,109,110,112,113,137,202,203,381) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pas3 = pd.read_csv(pas_path_3)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_1752\3828281856.py:4: DtypeWarning: Columns (41,42,43,131,132,133,213,214,435,444,451) have mixed types. Specify dtype option on import or set low_memory=Fa

In [8]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)
for i in df_pas1.columns:
    print(i)

Unnamed: 0
interview_date
MONTH
FinancialYear
ward
ward_n
ward_unique
SOA1
SOA2
BOROUGHNEIGHBOURHOOD
quarter
Quarter1.1
C2
NQ1
NQ2A
NQ2B
NQ2C
NQ2D
NQ2E
NQ2F
NQ2G
NQ2H
NQ2I
Q1
Q1D
Q3C
Q3F
Q3G
Q3H
Q3I
Q3J
Q3K
Q3L
Q126DA
Q126DB
Q126DC
NQ7A
NQ7B
Q10A
Q10B
Q10C
Q10D
Q10E
Q10F
Q10H
NQ10A
NQ10BA
NQ10BB
NQ10BC
NQ10BD
NQ10BE
NQ10BF
Q13
Q14A
Q14B
Q14C
Q14D
Q14E
Q15
Q16A
Q16B
Q16C
Q16D
Q16E
NQ21
Q21
Q54A
Q54B
Q54C
Q54D
Q54E
NQ57AA
NQ57AB
NQ57AC
NQ57AD
NQ57AE
NNQ27C
NNQ27E
Q37
Q38A
Q38B
Q38C
Q38D
Q38E
Q39A_2
Q39BA
Q39BB
Q39BC
Q39BD
Q39BE
NQ43
NQ44A
NQ44BA
NQ44BB
NQ44BC
NQ44BD
NQ44BE
NQ45A
NQ45BA
NQ45BB
NQ45BC
NQ45BD
NQ45BE
Q58
Q59
Q60
Q61
Q62A
Q62B
Q62C
Q62D
Q62E
Q62F
Q62TG
Q62H
Q62TI
Q62TJ
NQ62A
NQ62B
RQ80E
Q65
Q66
Legacy_Q73
Q79A
Q79B
Q79C
Q79D
Q79E
Q79F
Q79G
Q79H
Q79I
Q79J
NQ79A
NQ79ABA
NQ79ABB
NQ79ABC
NQ79ABD
NQ79ABE
Legacy_NQ79BA
NQ79BB
NQ79BC
NQ79BD
NQ79BI
SQ79A
SQ79B
SQ79C
SQ79DA
SQ79DB
SQ79DC
SQ79DD
SQ79DE
SQ79DF
SQ79E
SQ79F
NQ80
NQ80H
NQ80A
NQ80BA
NQ80BB
NQ80BC
NQ80BD
NQ80BE
NQ80C
NQ80D
B

In [7]:
questions_sas = ["XQ128A", "XQ128C", "XQ128D", "XQ128E", "XQ128G", "XQ128H", "XQ128F", "XQ128AA"]

In [12]:
df_pas1[["NQ119","PQ119","A120","A121"]].dropna()
#Q110 Have you been stopped?
#Q114 Have you been SaS'd or arrested
#Q119 Given the contacts with the police, satisfied?
#NQ119 Before being stopped and searched by police, how was opinion?
#PQ119 After, did opinion change?
#A120 Should police do SaS?
#A121 Do police use SaS fairly in your area?

,NQ119,PQ119,A120,A121
50,Mixed,Worse opinion,Tend to disagree,Fairly confident
56,Generally high,Unchanged,Tend to agree,Fairly confident
79,Generally high,Unchanged,Tend to agree,Fairly confident
115,Generally high,Better opinion,Strongly agree,Very confident
202,Generally high,Unchanged,Tend to agree,Fairly confident
...,...,...,...,...
25555,(spontaneous) No opinion,Unchanged,Tend to disagree,Not at all confident
25557,Mixed,Unchanged,Tend to agree,Fairly confident
25628,Mixed,Worse opinion,Strongly agree,Fairly confident
25631,Generally low,Better opinion,Neither agree nor disagree,Not at all confident


In [10]:
df_pas2[["Q136r","Q139r","ZQ110","ZNQ119","ZPQ119","A120","A121"]].dropna()
#["XQ119A","XQ119B","ZNQ119","ZPQ119"]
#ZQ110 Have you been stopped?
#ZNQ119 Before experience, what were opinions
#ZPQ119 After, how has opinion changed?
#A120 Should police do SaS?
#A121 Do police do SaS fairly in your area

,ZQ110,ZNQ119,ZPQ119
1,Yes,Generally high,Better opinion
54,Yes,Generally high,Unchanged
59,Yes,Generally high,Unchanged
98,Yes,(spontaneous) No opinion,Unchanged
110,Yes,Generally high,Unchanged
...,...,...,...
12556,Yes,Generally low,Unchanged
12601,Yes,Mixed,Better opinion
12616,Yes,Generally high,Unchanged
12684,Yes,Generally high,Better opinion


In [ ]:
df_pas3[["Q136r","Q139r","ZQ110","ZNQ119","ZPQ119","A120","A121"]].dropna()
#ZQ110 Have you been stopped?
#ZNQ119 Before experience, what were opinions
#ZPQ119 After, how has opinion changed?
#A120 Should police do SaS?
#A121 Do police do SaS fairly in your area

In [ ]:
df_pas4[["Q136r","Q139r","ZQ110","ZNQ119","ZPQ119","XQ128A","XQ128C","XQ128D", "A120","A121"]].dropna()
#ZQ110 Have you been stopped?
#ZNQ119 Before experience, what were opinions
#ZPQ119 After, opinions?
#XQ128A Have you been SaS'd?
#XQ128C Were they polite
#XQ128D Were they respectful
#A120 Agree that police should SaS
#A121 Do police do SaS fairly in your area

In [ ]:
df_pas5[["ReQ136","ReQ139","XQ128A","XQ128C","XQ128D","XQ128E","XQ128F","XQ128A2A","A120","A121"]]
#XQ128A Have you been SaS'd?
#XQ128C Were they polite?
#XQ128D Were they respectful?
#XQ128E Did they explain why?
#XQ128F Do you know someone who has been SaS
#XQ128A2A How was their experience?
#A120 Agree that police should SaS
#A121 Do police do SaS fairly in your area

In [ ]:
df_pas1["Q131"]

### New coding for questions:
SAS1: Have you been stopped? \
SAS11: Were they polite? \
SAS12: Were they respectful? \
SAS13: Did they explain why? \
SAS14: What was your opinion before being SaS \
SAS15: How did opinion change \
SAS2: Do you know someone that has been SaS'd \
SAS21: What is their opinion? \
SAS3: Should police do SAS? \
SAS31: Do you think they do SaS fairly in your area?


In [ ]:
df_pas1 = df_pas1.rename(columns= {"Q136r":"Age","Q139r":"Employment","NQ147r":"Ethnicity","Q114":"SAS1","NQ119":"SAS14","PQ119":"SAS15","A120":"SAS3", "A121":"SAS31", "NQ135BD": "MPS Trust", "Q60": "Good Job", "Q131": "Well informed"})
#Q114 Have you been SaS'd or arrested
#NQ119 Before being stopped and searched by police, how was opinion?
#PQ119 After, did opinion change?
#A120 Should police do SaS?
#A121 Do police use SaS fairly in your area?

In [ ]:
df_pas2 = df_pas2.rename(columns= {"Q136r":"Age","Q139r":"Employment","NQ147r":"Ethnicity","ZQ110":"SAS1","ZNQ119":"SAS14","ZPQ119":"SAS15","A120":"SAS3", "A121":"SAS31", "NQ135BD": "MPS Trust", "Q60": "Good Job", "Q131": "Well informed"})
#ZQ110 Have you been stopped?
#ZNQ119 Before experience, what were opinions
#ZPQ119 After, how has opinion changed?
#A120 Should police do SaS?
#A121 Do police do SaS fairly in your area

In [ ]:
df_pas3 = df_pas3.rename(columns= {"Q136r":"Age","Q139r":"Employment","NQ147r":"Ethnicity","ZQ110":"SAS1","ZNQ119":"SAS14","ZPQ119":"SAS15","A120":"SAS3", "A121":"SAS31", "NQ135BD": "MPS Trust", "Q60": "Good Job", "Q131": "Well informed"})
#ZQ110 Have you been stopped?
#ZNQ119 Before experience, what were opinions
#ZPQ119 After, how has opinion changed?
#A120 Should police do SaS?
#A121 Do police do SaS fairly in your area

In [ ]:
df_pas4 = df_pas4.rename(columns = {"Q136r":"Age","Q139r":"Employment","NQ147r":"Ethnicity","ZNQ119":"SAS14","ZPQ119":"SAS15","A120":"SAS3", "A121":"SAS31", "XQ128A": "SAS1", "XQ128C": "SAS11", "XQ128D": "SAS12", "NQ135BD": "MPS Trust", "Q60": "Good Job", "Q131": "Well informed"})
#ZQ110 Have you been stopped?
#ZNQ119 Before experience, what were opinions
#ZPQ119 After, opinions?
#XQ128A Have you been SaS'd?
#XQ128C Were they polite
#XQ128D Were they respectful
#A120 Agree that police should SaS
#A121 Do police do SaS fairly in your area

In [ ]:
df_pas5 = df_pas5.rename(columns={"ReQ136":"Age","ReQ139":"Employment","ReNQ147":"Ethnicity","ZNQ119":"SAS14","ZPQ119":"SAS15","A120":"SAS3", "A121":"SAS31", "XQ128A": "SAS1", "XQ128C": "SAS11", "XQ128D": "SAS12", "XQ128E": "SAS13", "XQ128F": "SAS2", "XQ128A2A": "SAS21", "NQ135BD": "MPS Trust", "Q60": "Good Job", "Q131": "Well informed"})
#XQ128A Have you been SaS'd?
#XQ128C Were they polite?
#XQ128D Were they respectful?
#XQ128E Did they explain why?
#XQ128F Do you know someone who has been SaS
#XQ128A2A How was their experience?
#A120 Agree that police should SaS
#A121 Do police do SaS fairly in your area

In [ ]:
df_pas1 = df_pas1.rename(columns={"C2":"Borough"})
df_pas2 = df_pas2.rename(columns={"C2":"Borough"})
df_pas3 = df_pas3.rename(columns={"C2":"Borough"})
df_pas4 = df_pas4.rename(columns={"C2":"Borough"})

In [ ]:
df_pas2["FinancialYear"] = "2017/18"
df_pas3["FinancialYear"] = "2018/19"
df_pas4["FinancialYear"] = "2019/20"
df_pas5["FinancialYear"] = "2020/21"

In [ ]:
df_pas_sas1 = df_pas1[["Age","Employment","Ethnicity","FinancialYear","Borough","SAS1","SAS14","SAS15","SAS3","SAS31", "MPS Trust", "Good Job", "Well informed"]]

In [ ]:
df_pas_sas2 = df_pas2[["Age","Employment","Ethnicity","FinancialYear","Borough","SAS1","SAS14","SAS15","SAS3","SAS31"]]

In [ ]:
df_pas_sas3 = df_pas3[["Age","Employment","Ethnicity","FinancialYear","Borough","SAS1","SAS14","SAS15","SAS3","SAS31"]]

In [ ]:
df_pas_sas4 = df_pas4[["Age","Employment","Ethnicity","FinancialYear","Borough","SAS14","SAS15","SAS3","SAS31","SAS1","SAS11","SAS12"]]

In [ ]:
df_pas_sas5 = df_pas5[["Age","Employment","Ethnicity","FinancialYear","Borough","SAS3","SAS31","SAS1","SAS11","SAS12","SAS13","SAS2","SAS21"]]

In [ ]:
df_pas_sas = pd.concat([df_pas_sas1, df_pas_sas2, df_pas_sas3, df_pas_sas4, df_pas_sas5], ignore_index=True)

In [ ]:
df_pas_sas.columns

In [ ]:
df_pas_sas.to_csv("PAS_stop_and_search.csv")

In [ ]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)
df_pas_sas[df_pas_sas["SAS1"] == "Yes"]

In [ ]:
count_df = df_pas_sas.groupby(['Borough', 'SAS1']).size().unstack(fill_value=0)
count_df['Total'] = count_df['Yes'] + count_df['No']
count_df['Percentage_Yes'] = (count_df['Yes'] / count_df['Total']) * 100

# Create the final DataFrame with the desired information
sas_count_in_pas = count_df[['Total', 'Percentage_Yes']].reset_index()

# Ensure the 'Borough' column in sas_count_in_pas matches the 'LAD11NM' column in the GeoDataFrame
# For demonstration, let's assume 'Borough' and 'LAD11NM' have matching names

# Merge the GeoDataFrame with the data DataFrame
merged_gdf = shape.merge(sas_count_in_pas, left_on='LAD11NM', right_on='Borough')

# Plot the map with the percentage of 'Yes' instances
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
merged_gdf.plot(column='Percentage_Yes', cmap='coolwarm', linewidth=0, ax=ax, edgecolor='1', legend=True)

plt.title('Percentage of people being SaSd per borough')
plt.show()

In [ ]:
for year in df_pas_sas['FinancialYear'].unique():
    # Filter the DataFrame for the current year
    df_year = df_pas_sas[df_pas_sas['FinancialYear'] == year]

    # Group by Borough and SAS1, count occurrences
    count_df = df_year.groupby(['Borough', 'SAS1']).size().unstack(fill_value=0)
    count_df['Total'] = count_df['Yes'] + count_df['No']
    count_df['Percentage_Yes'] = (count_df['Yes'] / count_df['Total']) * 100

    # Create the final DataFrame with the desired information
    sas_count_in_pas = count_df[['Total', 'Percentage_Yes']].reset_index()

    # Merge the GeoDataFrame with the data DataFrame
    merged_gdf = shape.merge(sas_count_in_pas, left_on='LAD11NM', right_on='Borough')

    # Plot the map with the percentage of 'Yes' instances
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    merged_gdf.plot(column='Percentage_Yes', cmap='coolwarm', linewidth=0, ax=ax, edgecolor='1', legend=True)

    plt.title(f'Percentage of people being SaSd per borough in {year}')
    plt.show()

In [ ]:
sas_count_in_pas